In [5]:
import pandas as pd 
import numpy as np 
import random

In [6]:
"""
This function checks if the given values of x, y and z satisfy the condition x+y+z==10 and all of them are greater than 0.

@param x: a numeric value
@param y: a numeric value
@param z: a numeric value
@return: Returns True if the condition is satisfied, False otherwise.
"""

def objective_funtion(x,y,z):
    if (x+y+z == 10 and x>0 and y> 0 and z>0):
        return True
    else:
        return False

In [230]:

"""
This function performs a mutation on the input child_gene string with a probability of 1/20.
If a mutation occurs, a random bit in the gene is flipped.
Returns the mutated child_gene string or the original child_gene string if no mutation occurs.


"""

def mutaion_function(child_gene):
    
    random_number = random.randint(1 ,20)
    if(random_number == 1):
        print("Mutation occured !!")
        random_bit = random.randint(1 , len(child_gene))
        child_gene = list(child_gene)
        random_bit = random.randint(0 , len(child_gene)-1)
        if(child_gene[random_bit] == "0"):
            child_gene[random_bit] = "1"
        else:
            child_gene[random_bit] ="0"
        return(''.join(map(str , child_gene)))  
    else:
        return(child_gene)  

In [171]:
"""
This code contains two functions: objective_function and fitness.

The objective_function takes three arguments x, y, and z and returns their product with predefined constants: 20000, 15000, and 17000.

The fitness function takes a DataFrame as an argument, parents_dataframe, which contains
values for x, y, and z. 
The function calculates the fitness value of each record in the DataFrame by:

1- Summing the results of objective_function for each record in the DataFrame. 
This sum is assigned to the variable fitness_total.

2 - Appending a new column, fitness, to the DataFrame.
 
3 - This column is populated with the fitness value of each record, which is calculated by dividing 1 by the 
result of objective_function for each record divided by fitness_total.

4 - Sorting the DataFrame in descending order by the fitness column.
Returning the sorted DataFrame.
"""

def objective_function(x , y , z):
    if (x+y+z == 10 and x>0 and y>0 and z>0):
        
        return (x*20000 + y*15000 + z*17000)
 
    else:
       
        return 0
def fitness(parents_dataframe):

    fitness_total = sum(parents_dataframe.apply(lambda x: objective_function(x['x'],x['y'],x['z']) , axis=1))
    parents_dataframe["fitness"] = parents_dataframe.apply(lambda x : 1/(objective_function(x['x'],x['y'],x['z'])/fitness_total) if (objective_function(x['x'],x['y'],x['z']) > 0) else 0 , axis=1)
  
    parents_dataframe = parents_dataframe[parents_dataframe["fitness"]>0]
    parents_dataframe  = parents_dataframe.sort_values(by=['fitness'] , ascending=False)
    return (parents_dataframe.reset_index(drop=True))


In [43]:
"""
This function generates a population of parent individuals based on a given objective function.

Inputs:
- objective_function: a function that takes three integer inputs (x, y, z) and
returns a boolean indicating whether or not the input values are valid for the population

Output:
- generation: a list of dictionaries containing the x, y, and z values for each 
parent individual in the population
"""
def generate_population():
    
    number_of_parents = 4
    generation =[]
    while (len(generation) < number_of_parents):
        x =(random.randint(1,9))
        y =(random.randint(1,9))
        z =(random.randint(1,9))
        if (objective_funtion(x , y , z)):
            temp = {
                'x':x,
                'y':y ,
                'z':z
            }
            generation.append(temp)
    
    df = pd.DataFrame(generation)
   
    return df





,x,y,z,fitness,generation
0,1,6,3,4.273292,Gen_1
1,3,5,2,4.071006,Gen_1
2,1,1,8,4.023392,Gen_1
3,7,2,1,3.679144,Gen_1


In [113]:
def convert_to_binary(x):
    return (bin(x)[2:].zfill(4))

def encoding_function(parents_dataframe):
    temp_dataframe = parents_dataframe.drop(['fitness' , 'generation'],axis=1)
    column_list = temp_dataframe.columns
    for columns in column_list:
        temp_dataframe[columns]  = temp_dataframe[columns].apply(lambda x : convert_to_binary(x))
    
    return(temp_dataframe)

def decoding_function(x):
    return int(x ,2)
    

In [125]:
def bit_swap_function(parent_column):
    elite_parent_counter = 0 
    parent_counter = 1
    child_list =[]
    while(elite_parent_counter <= len(parent_column)-1):
        elite_parent = parent_column[elite_parent_counter]
        
        for parent in parent_column[parent_counter:]:
            child_one = list(elite_parent)
            child_two = list(parent)
            gene_one = elite_parent[1:3]
            gene_two = parent[1:3]
            child_one[1:3] = gene_two
            child_two[1:3] = gene_one
            child_one = ''.join(map(str , child_one))
            child_two =''.join(map(str , child_two))
            child_one =(mutaion_function(child_one))
            child_two=(mutaion_function(child_two))
            child_list.append(child_one)
            child_list.append(child_two)  
        elite_parent_counter = elite_parent_counter + 1
        parent_counter = parent_counter + 1 
            
   
    print("child_list --->",child_list)
    return child_list
    

def cross_over_function(parent_dataframe):
    encoded_dataframe = encoding_function(parent_dataframe)
    encoded_dataframe_new = pd.DataFrame()
    for column in encoded_dataframe.columns:
        encoded_dataframe_new[column] = bit_swap_function(list(encoded_dataframe[column]))
    # the rows are arranged in order of the fitness , the top row has the highest fitness
    return encoded_dataframe_new.applymap(decoding_function)
    

In [283]:
start_generation = fitness(generate_population())
start_generation['generation'] ="Gen_1"
print(start_generation)


new_generation = cross_over_function(start_generation)
new_generation.drop_duplicates(inplace=True)
new_generation['generation'] ="Gen_2"
new_generation['fitness']=''
new_generation = pd.concat([start_generation , new_generation] , axis=0)
print((fitness(new_generation)))
# print(fitness(start_generation , "Gen_2"))

   x  y  z   fitness generation
0  1  8  1  4.280255      Gen_1
1  1  3  6  4.023952      Gen_1
2  3  4  3  3.929825      Gen_1
3  3  1  6  3.796610      Gen_1
Mutation occured !!
Mutation occured !!
child_list ---> ['0001', '0001', '0011', '0001', '0001', '0011', '0011', '0001', '0011', '0001', '0011', '0011']
Mutation occured !!
child_list ---> ['1010', '0001', '1100', '0000', '1000', '0001', '0101', '0010', '1001', '0011', '0000', '0101']
child_list ---> ['0111', '0000', '0011', '0001', '0111', '0000', '0010', '0111', '0110', '0110', '0111', '0010']
   x  y  z   fitness generation
0  1  8  1  7.496815      Gen_1
1  1  3  6  7.047904      Gen_1
2  1  3  6  7.047904      Gen_2
3  3  5  2  6.964497      Gen_2
4  1  2  7  6.964497      Gen_2
5  3  4  3  6.883041      Gen_1
6  3  1  6  6.649718      Gen_1
